# Profiles for Allocation

-Christian C. Feb/2020 (With majority of code being reused from Stack exchange and Google)

-Functionality: This module puts together csv created in previous module. And creates final report
(cleans data, aligns naming, creates templates, and puts all info together in final data frame/report).

-Note: Kept it for the moment in 2 separate modules for readability.


## Imports Libraries

In [1]:
import os
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

In [2]:
path = "C:\\Users\\CAS233\\Notebooks Christian\\DS_importing wave2\\Dataframes"
os.chdir(path)

In [3]:
#basepath = path
#for entry in os.listdir(basepath):
#    if os.path.isfile(os.path.join(basepath, entry)):
#        print(entry)

In [4]:
#print(df_ds4.shape)
#print(df_ds4['ND QTY'].sum())
#print(5682547+7252989+794987)
#print("*********")
#print(df_ds1.shape)
#print(df_ds1['ND QTY'].sum())
#print(21385565+3243461)
#print("*********")
#print(df_SGMT.shape)
#print(df_conso.shape)
#print(df_waterfall.shape)

## Imports CSV base files into separate dataframes (1 df per csv)

In [5]:
input_files = ['df_ds1', 'df_ds4','df_SGMT', 'df_waterfall','df_conso']

for my_file in input_files:
 
    df = pd.read_csv(my_file+".csv")
    df.dropna(how='all', inplace=True)
    df.drop(columns=['Unnamed: 0'], inplace=True)
  
    exec(str(my_file)+" = df")
   

C:\Users\CAS233\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df = pd.read_csv('df_MFP.csv', converters={'PC5': lambda x: str(x)})
df_MFP = df

df = pd.read_csv('df_PG.csv')
df_pg = df



In [7]:
#Filters Months. IT sends now downloads for full horizon so filter as you wish here
my_months_list =['2020.06', '2020.07', '2020.08', '2020.09', '2020.10', '2020.11', '2020.12', '2021.01']
df_ds1 = df_ds1[df_ds1.FISCAL.isin(my_months_list)]
df_ds4 = df_ds4[df_ds4.FISCAL.isin(my_months_list)]


## Align field naming convention between dataframes

In [8]:
df_conso.rename({'Colorway Code': 'PC9', 'Product Code': 'PC5'}, axis='columns',inplace=True)
df_SGMT.rename({'Planning Group': 'PG'}, axis='columns',inplace=True)
df_waterfall.rename({'Account': 'PG', 'Material': 'PC9', 'SumOfQtyTotal': 'QTY_WF'}, axis='columns',inplace=True)
df_ds4.rename({'PG DESC': 'PG'},axis='columns',inplace=True)

#Drops unneeded columns from df waterfall. For cleaner code this could happen on previous module.
df_waterfall.drop(columns=['SalesOffice','Channel','Add/Drop'], axis=1,inplace=True)

#Gets rid of spaces in sizes. For cleaner code this could happen on previous module.
df_SGMT.W = df_SGMT.W.str.strip()
df_SGMT.L = df_SGMT.L.str.strip()


In [22]:
list_0 = df_pg['PG'].unique()
list_1 = df_waterfall['PG'].unique()
list_2 = df_SGMT['PG'].unique()
list_3 = df_ds4['PG'].unique()
print(list(set(list_0).difference(list_1)))
print(list(set(list_0).difference(list_2)))
print(list(set(list_0).difference(list_3)))


[]
['LSE OUTLET UK', 'LSE OUTLET EU', 'TURKEY OUTLET']
['LSE OUTLET UK', 'LSE OUTLET EU', 'TURKEY OUTLET']


## Creates Dataframe "df_basis"

In [10]:
#Create df_basis. Conso is the start of it. SGMT next. MFP next. PG next.
df_basis = pd.merge(df_conso,df_SGMT, on='PC9', how='left')
df_basis = pd.merge(df_basis,df_waterfall, on=['PC9','PG'], how='left')
df_basis = pd.merge(df_basis,df_MFP, on=['PC5'], how='left')
df_basis = pd.merge(df_basis,df_pg, on=['PG'], how='left')

## Prepares DS1 and DS4 dataframes for creation of Profile Templates

In [11]:
#adds MPF to DS1
df_ds1["PC5"] = df_ds1["PC9"].str[:5]
df_ds1 = pd.merge(df_ds1,df_MFP, on=['PC5'], how='left')

#adds MFP to DS4
df_ds4["PC5"] = df_ds4["PC9"].str[:5]
df_ds4 = pd.merge(df_ds4,df_MFP, on=['PC5'], how='left')

In [12]:
#adds W and L to DS1
new = df_ds1["SZ"].str.split(" +", n = 1, expand = True)
new.rename(columns={0: "W", 1: "L"}, inplace=True)
new.L.fillna(value=pd.np.nan, inplace=True)
new['L'].fillna('-', inplace=True)
df_ds1["W"]= new['W'] 
df_ds1["L"]= new['L'] 

#adds W and L to DS4
new = df_ds4["SZ"].str.split(" +", n = 1, expand = True)
new.rename(columns={0: "W", 1: "L"}, inplace=True)
new.L.fillna(value=pd.np.nan, inplace=True)
new['L'].fillna('-', inplace=True)
df_ds4["W"]= new['W'] 
df_ds4["L"]= new['L'] 

## Creates Profile Templates (ie. profiles by pc9, pc5, MFP; from DS1 and DS4)

In [13]:
#ds1 pg pc9
df_a = df_ds1.groupby(['CHANNEL','AFFILIATE','PC9','W','L'], as_index=False)['ND QTY'].sum()
df_b = df_ds1.groupby(['CHANNEL','AFFILIATE','PC9'], as_index=False)['ND QTY'].sum()

df = pd.merge(df_a, df_b, on=['PC9','CHANNEL','AFFILIATE'], how='left')
df['ND QTY_x'] = df['ND QTY_x'].astype(float) 
df['ND QTY_y'] = df['ND QTY_y'].astype(float) 
df['df_ds1_pc9'] = df['ND QTY_x']/df['ND QTY_y']

df_ds1_pc9 = df

#ds4 pg pc9
df_a = df_ds4.groupby(['PG','PC9','W','L'], as_index=False)['ND QTY'].sum()
df_b = df_ds4.groupby(['PG','PC9'], as_index=False)['ND QTY'].sum()

df = pd.merge(df_a, df_b, on=['PC9','PG'], how='left')
df['ND QTY_x'] = df['ND QTY_x'].astype(float) 
df['ND QTY_y'] = df['ND QTY_y'].astype(float) 
df['df_ds4_pc9'] = df['ND QTY_x']/df['ND QTY_y']

df_ds4_pc9 = df



In [14]:
#ds1 ch/aff pc5
df_a = df_ds1.groupby(['CHANNEL','AFFILIATE','PC5','W','L'], as_index=False)['ND QTY'].sum()
df_b = df_ds1.groupby(['CHANNEL','AFFILIATE','PC5'], as_index=False)['ND QTY'].sum()

df = pd.merge(df_a, df_b, on=['PC5','CHANNEL','AFFILIATE'], how='left')
df['ND QTY_x'] = df['ND QTY_x'].astype(float) 
df['ND QTY_y'] = df['ND QTY_y'].astype(float) 
df['df_ds1_pc5'] = df['ND QTY_x']/df['ND QTY_y']

df_ds1_pc5 = df

#ds4 pg pc5
df_a = df_ds4.groupby(['PG','PC5','W','L'], as_index=False)['ND QTY'].sum()
df_b = df_ds4.groupby(['PG','PC5'], as_index=False)['ND QTY'].sum()

df = pd.merge(df_a, df_b, on=['PC5','PG'], how='left')
df['ND QTY_x'] = df['ND QTY_x'].astype(float) 
df['ND QTY_y'] = df['ND QTY_y'].astype(float) 
df['df_ds4_pc5'] = df['ND QTY_x']/df['ND QTY_y']

df_ds4_pc5 = df

#ds1 ch/aff mfp
df_a = df_ds1.groupby(['CHANNEL','AFFILIATE','MFP','W','L'], as_index=False)['ND QTY'].sum()
df_b = df_ds1.groupby(['CHANNEL','AFFILIATE','MFP'], as_index=False)['ND QTY'].sum()

df = pd.merge(df_a, df_b, on=['MFP','CHANNEL','AFFILIATE'], how='left')
df['ND QTY_x'] = df['ND QTY_x'].astype(float) 
df['ND QTY_y'] = df['ND QTY_y'].astype(float) 
df['df_ds1_mfp'] = df['ND QTY_x']/df['ND QTY_y']

df_ds1_mfp = df

#ds4 ch/aff mfp
df_a = df_ds4.groupby(['PG','MFP','W','L'], as_index=False)['ND QTY'].sum()
df_b = df_ds4.groupby(['PG','MFP'], as_index=False)['ND QTY'].sum()

df = pd.merge(df_a, df_b, on=['MFP','PG'], how='left')
df['ND QTY_x'] = df['ND QTY_x'].astype(float) 
df['ND QTY_y'] = df['ND QTY_y'].astype(float) 
df['df_ds4_mfp'] = df['ND QTY_x']/df['ND QTY_y']

df_ds4_mfp = df


#ds1 lse pc5
df_a = df_ds1.groupby(['PC5','W','L'], as_index=False)['ND QTY'].sum()
df_b = df_ds1.groupby(['PC5'], as_index=False)['ND QTY'].sum()

df = pd.merge(df_a, df_b, on=['PC5'], how='left')
df['ND QTY_x'] = df['ND QTY_x'].astype(float) 
df['ND QTY_y'] = df['ND QTY_y'].astype(float) 
df['df_ds1_pc5_lse'] = df['ND QTY_x']/df['ND QTY_y']

df_ds1_pc5_lse = df


#ds1 lse mfp
df_a = df_ds1.groupby(['MFP','W','L'], as_index=False)['ND QTY'].sum()
df_b = df_ds1.groupby(['MFP'], as_index=False)['ND QTY'].sum()

df = pd.merge(df_a, df_b, on=['MFP'], how='left')
df['ND QTY_x'] = df['ND QTY_x'].astype(float) 
df['ND QTY_y'] = df['ND QTY_y'].astype(float) 
df['df_ds1_mfp_lse'] = df['ND QTY_x']/df['ND QTY_y']

df_ds1_mfp_lse = df


## Appends templates to "basis_df"

In [15]:
df_basis = pd.merge(df_basis,df_ds1_pc9, on=['PC9','W','L','CHANNEL','AFFILIATE'], how='left')
df_basis = pd.merge(df_basis,df_ds4_pc9, on=['PC9','W','L','PG'], how='left')

df_basis = pd.merge(df_basis,df_ds1_pc5, on=['PC5','W','L','CHANNEL','AFFILIATE'], how='left')
df_basis = pd.merge(df_basis,df_ds4_pc5, on=['PC5','W','L','PG'], how='left')

df_basis = pd.merge(df_basis,df_ds1_mfp, on=['MFP','W','L','CHANNEL','AFFILIATE'], how='left')
df_basis = pd.merge(df_basis,df_ds4_mfp, on=['MFP','W','L','PG'], how='left')

df_basis = pd.merge(df_basis,df_ds1_pc5_lse, on=['PC5','W','L'], how='left')
df_basis = pd.merge(df_basis,df_ds1_mfp_lse, on=['MFP','W','L'], how='left')

## Data cleaning to "basis_df"

In [16]:
#Clean Nan and remove unnecessary columns created during merges
df_basis = df_basis.fillna(0)
df_basis = df_basis.drop(['ND QTY_x_x','ND QTY_x_y', 'ND QTY_y_x', 'ND QTY_y_y'], axis=1)


In [17]:
#Sort order of columns (only for personal preference)
df_basis = df_basis[['CHANNEL','AFFILIATE', 'PG',
                     'PC9', 'PC5', 'QTY_WF',
                    'Add/Drop','Department','Mainline/OPP/Concepts','LSE Merch Story', 'MFP categorization','MFP',
                    'Product Name','New or C/O or D0','Sizes','Size Grid Name',
                    'Size','Selected','W','L',
                      'df_ds4_pc9','df_ds4_pc5','df_ds4_mfp',
                      'df_ds1_pc9','df_ds1_pc5','df_ds1_mfp',
                      'df_ds1_pc5_lse','df_ds1_mfp_lse'
                     ]]

In [18]:
#Filter rows
#I only want to see records with Forecast > 0 from waterfall, that are not dropped in conso, 
#and only the PGs selected below.
filter_1 = df_basis['QTY_WF'] > 0
filter_2 = df_basis['Add/Drop'] != "DROP"
filter_list_3 = ['NORDICS RETAIL','EE RETAIL','FRANCE RETAIL','CENTRAL RETAIL',
                'ITALY RETAIL', 'ITALY KEY ACCOUNTS',
                 'RUSSIA RETAIL',
                 'SPAIN RETAIL', 'EL CORTE INGLES CONCESSION',
                 'TURKEY RETAIL','BOYNER TURKEY CONCESSION',
                 'NORTH RETAIL',
                 'ECOM EU','ECOM UK',
                'GALERIES LAFAYETTE','PRINTEMPS']

#Copies a subsett of df_basis, and assigns it to new dataframe df_basis2
#i kept a copy for easier debugging from my side.
df_basis2 = df_basis[(filter_1) & (filter_2)]
df_basis2 = df_basis2[df_basis2.PG.isin(filter_list_3)]
df_basis2 = df_basis2.sort_values(by=['CHANNEL', 'AFFILIATE','PG','PC9','W','L'])



In [19]:
#Creates new fields in df_basis 2, to keep track final profile, from which template is it grabbing it,
#and flagging at the end of the exercise if a profile is still missing.
df_basis2['PROFILE'] = 0.00
df_basis2['LVL'] = 0
df_basis2['STILL_MISSING'] = 0


In [20]:
#Populates column with Final profile (the column that the final users at Allocation team will look at).
#and puts a flag from 1 to 8, to know later from which template it grabbed the profile from.

#set initial value
df_basis2['PROFILE'] = df_basis2['df_ds4_pc9']
df_basis2['LVL'] = 1

#Start filling blanks
mask = df_basis2['PROFILE'] == 0
df_basis2['PROFILE'][mask] = df_basis2['df_ds4_pc5']
df_basis2['LVL'][mask] = 2

mask = df_basis2['PROFILE'] == 0
df_basis2['PROFILE'][mask] = df_basis2['df_ds4_mfp']
df_basis2['LVL'][mask] = 3


mask = df_basis2['PROFILE'] == 0
df_basis2['PROFILE'][mask] = df_basis2['df_ds1_pc9']
df_basis2['LVL'][mask] = 4

mask = df_basis2['PROFILE'] == 0
df_basis2['PROFILE'][mask] = df_basis2['df_ds1_pc5']
df_basis2['LVL'][mask] = 5

mask = df_basis2['PROFILE'] == 0
df_basis2['PROFILE'][mask] = df_basis2['df_ds1_mfp']
df_basis2['LVL'][mask] = 6


mask = df_basis2['PROFILE'] == 0
df_basis2['PROFILE'][mask] = df_basis2['df_ds1_pc5_lse']
df_basis2['LVL'][mask] = 7


mask = df_basis2['PROFILE'] == 0
df_basis2['PROFILE'][mask] = df_basis2['df_ds1_mfp_lse']
df_basis2['LVL'][mask] = 8

mask = df_basis2['PROFILE'] == 0
df_basis2['STILL_MISSING'][mask] = 1





C:\Users\CAS233\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\CAS233\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\CAS233\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/inde

In [24]:
#Exports final output to CSV. Then to give formatting for allocation team to use.
#df_basis2.to_csv("df_basis_output.csv")
#df_basis2.to_excel("df_basis_output_xl.xlsx")